In [ ]:
#ROUGE & BERTScore

REST API endpoint at http://127.0.0.1:8000/summarize/ where you can POST articles and receive summaries.






In [ ]:
from evaluate import load
rouge = load("rouge")
bert_score = load("bertscore")
rouge_results = rouge.compute(predictions=preds, references=refs)
bert_results = bert_score.compute(predictions=preds, references=refs, lang="en")
print(f"ROUGE-L: {rouge_results['rougeL']}, BERTScore F1: {bert_results['f1']}")

Example: Latency Measurement

import time
start = time.time()
_ = model.generate(**inputs)
end = time.time()
latency = (end - start) / len(inputs["input_ids"])
print(f"Average latency per summary: {latency:.4f} seconds")


summarization_metrics.ipynb

*** code ****

pip install transformers datasets evaluate bert-score rouge_score
from transformers import pipeline
from datasets import load_dataset
from evaluate import load
from bert_score import score
from rouge_score import rouge_scorer
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

dataset = load_dataset("cnn_dailymail", version="3.0.0", split="test[:1%]")

predictions = []
for article in dataset["article"]:
    summary = summarizer(article, max_length=150, min_length=50, do_sample=False)
    predictions.append(summary[0]["summary_text"])

rouge_metric = load("rouge")
rouge_results = rouge_metric.compute(predictions=predictions, references=dataset["highlights"])
print(f"ROUGE Score: {rouge_results}")

P, R, F1 = score(predictions, dataset["highlights"], lang="en")
print(f"BERTScore F1: {F1.mean():.4f}")
scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)
rouge_l_scores = [scorer.score(ref, pred)["rougeL"].fmeasure for ref, pred in zip(dataset["highlights"], predictions)]
average_rouge_l = sum(rouge_l_scores) / len(rouge_l_scores)
print(f"Average ROUGE-L F1: {average_rouge_l:.4f}")

bitnet_summarization_deployment.ipynb

pip install transformers bitnet

from bitnet import BitNetForCausalLM, BitNetTokenizer

model = BitNetForCausalLM.from_pretrained("microsoft/bitnet-b1_58-2B")
tokenizer = BitNetTokenizer.from_pretrained("microsoft/bitnet-b1_58-2B")
# Define Summarization Function
def summarize_text(text, model, tokenizer, max_length=150):
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs["input_ids"], max_length=max_length, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

article = "Your long article text here..."
summary = summarize_text(article, model, tokenizer)
print("Summary:", summary)
pip install fastapi uvicorn
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class Article(BaseModel):
    text: str

@app.post("/summarize/")
def create_summary(article: Article):
    summary = summarize_text(article.text, model, tokenizer)
    return {"summary": summary}
uvicorn app:app –reload
